In [1]:
# Origem: Centro de Curitiba (longitude, latitude)
origem = ["-49.2733,-25.4284"]

# Destinos: Outras localidades dentro de Curitiba
destinos = [
    "-49.2788,-25.4422",  # Batel
    "-49.2948,-25.4745",  # Portão
    "-49.2491,-25.4925",  # Santa Felicidade
    "-49.1777,-25.5195"   # Boqueirão
]

# Todas as localidades (origem + destinos)
localidades = origem + destinos

In [2]:
import requests

def get_distance_matrix(localidades):
    """
    Obtém a matriz de distâncias e tempos entre as localidades usando o OSRM.
    """
    url = "http://router.project-osrm.org/table/v1/driving/"
    coordinates = ";".join(localidades)
    params = {
        "sources": ";".join(map(str, range(len(localidades)))),  # Todas as origens
        "destinations": ";".join(map(str, range(len(localidades)))),  # Todos os destinos
        "annotations": "duration,distance"
    }
    response = requests.get(url + coordinates, params=params)
    if response.status_code == 200:
        data = response.json()
        duration_matrix = data['durations']  # Matriz de tempos em segundos
        distance_matrix = data['distances']  # Matriz de distâncias em metros
        return duration_matrix, distance_matrix
    else:
        print("Erro ao obter a matriz de distâncias:", response.status_code)
        return None, None
    
# Obtém a matriz de distâncias e tempos
duration_matrix, distance_matrix = get_distance_matrix(localidades)

# Exibe a matriz de tempos
if duration_matrix:
    print("Matriz de tempos (segundos):")
    for row in duration_matrix:
        print(row)

# Exibe a matriz de distâncias
if distance_matrix:
    print("Matriz de distâncias (metros):")
    for row in distance_matrix:
        print(row)

Matriz de tempos (segundos):
[0, 312.8, 728.6, 869.6, 1393.4]
[400.3, 0, 481, 782.4, 1508.4]
[792.9, 539.7, 0, 521.4, 1493.4]
[884.1, 638.5, 637.3, 0, 1105.6]
[1373.5, 1351.4, 1523.5, 1022.4, 0]
Matriz de distâncias (metros):
[0, 2304.1, 7082.5, 8617, 18219.6]
[3120, 0, 5044.4, 8133.5, 19071]
[7872.8, 5691.1, 0, 6283, 18382.3]
[8711, 6702.8, 7060.8, 0, 12166.3]
[16333.5, 16507.5, 19197.8, 12007.8, 0]


In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Passo 3: Resolver o problema de roteamento com a OR-Tools
# Número de funcionários (veículos)
num_funcionarios = 1  # Apenas 1 veículo

# Localização do depósito (origem: Centro de Curitiba)
depot = 0

# Cria o gerenciador de índices e o modelo de roteamento
manager = pywrapcp.RoutingIndexManager(len(localidades), num_funcionarios, depot)
routing = pywrapcp.RoutingModel(manager)

# Define os pesos para tempo e distância (ajuste conforme necessário)
peso_tempo = 0.7  # Peso para o tempo
peso_distancia = 0.3  # Peso para a distância

# Função de custo combinada (tempo e distância)
def combined_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    tempo = duration_matrix[from_node][to_node]  # Tempo entre from_node e to_node
    distancia = distance_matrix[from_node][to_node]  # Distância entre from_node e to_node
    return (peso_tempo * tempo) + (peso_distancia * distancia)  # Custo combinado

# Registra a função de custo combinada
combined_callback_index = routing.RegisterTransitCallback(combined_callback)

# Associa a função de custo combinada ao modelo
routing.SetArcCostEvaluatorOfAllVehicles(combined_callback_index)

# Configura parâmetros de busca
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Resolve o problema
solution = routing.SolveWithParameters(search_parameters)

# Exibe a solução
if solution:
    print(f'Custo total: {solution.ObjectiveValue()}')
    for vehicle_id in range(num_funcionarios):
        index = routing.Start(vehicle_id)
        plan_output = f'Rota do funcionário {vehicle_id}:\n'
        route_time = 0
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f'{localidades[manager.IndexToNode(index)]} -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            from_node = manager.IndexToNode(previous_index)
            to_node = manager.IndexToNode(index)
            route_time += duration_matrix[from_node][to_node]
            route_distance += distance_matrix[from_node][to_node]
        plan_output += f'{localidades[manager.IndexToNode(index)]}\n'
        plan_output += f'Tempo da rota: {route_time} segundos\n'
        plan_output += f'Distância da rota: {route_distance} metros\n'
        print(plan_output)
else:
    print('Nenhuma solução encontrada.')

load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Program Files (x86)\Eduardo\code\hub_ia\Trabalhos\venv\Lib\site-packages\ortools\.libs\ortools.dll...
Custo total: 0
Rota do funcionário 0:
-49.2733,-25.4284 -> -49.1777,-25.5195 -> -49.2491,-25.4925 -> -49.2948,-25.4745 -> -49.2788,-25.4422 -> -49.2733,-25.4284
Tempo da rota: 3993.1000000000004 segundos
Distância da rota: 46

In [4]:
def get_route(coordinates):
    """
    Obtém a rota real entre uma lista de coordenadas usando o OSRM.
    Retorna uma lista de coordenadas (latitude, longitude) que representam a rota.
    """
    url = "http://router.project-osrm.org/route/v1/driving/"
    coordinates_str = ";".join(coordinates)
    params = {
        "overview": "full",  # Retorna a geometria completa da rota
        "geometries": "geojson"  # Formato das coordenadas
    }
    response = requests.get(url + coordinates_str, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 'Ok':
            return data['routes'][0]['geometry']['coordinates']  # Lista de coordenadas (longitude, latitude)
    return None

In [5]:
import folium
from folium.plugins import PolyLineTextPath

# Cria um mapa centrado no Centro de Curitiba
m = folium.Map(location=[-25.4284, -49.2733], zoom_start=12)

localidades = [f"{lat},{lon}" for lon, lat in [coord.split(",") for coord in localidades]]
destinos = [f"{lat},{lon}" for lon, lat in [coord.split(",") for coord in destinos]]
origem = [f"{lat},{lon}" for lon, lat in [coord.split(",") for coord in origem]]

# Adiciona a origem ao mapa
for i, loc in enumerate(origem):
    lat, lon = map(float, loc.split(','))
    folium.Marker(
        [lat, lon],
        tooltip=f'Centro de Curitiba {i+1}',
        icon=folium.Icon(color='green')
    ).add_to(m)

# Adiciona os destinos ao mapa
for i, loc in enumerate(destinos):
    lat, lon = map(float, loc.split(','))
    folium.Marker(
        [lat, lon],
        tooltip=f'Destino {i+1}'
    ).add_to(m)

# Adiciona a rota real ao mapa com setas indicando o sentido
if solution:
    for vehicle_id in range(num_funcionarios):
        index = routing.Start(vehicle_id)
        route_coords = []
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            lat, lon = map(float, localidades[node].split(','))
            route_coords.append(f"{lon},{lat}")  # Formato longitude,latitude
            index = solution.Value(routing.NextVar(index))
        # Adiciona a última localidade
        node = manager.IndexToNode(index)
        lat, lon = map(float, localidades[node].split(','))
        route_coords.append(f"{lon},{lat}")

        # Obtém a rota real entre os pontos
        route = get_route(route_coords)
        if route:
            # Converte as coordenadas para o formato [latitude, longitude]
            route_lat_lon = [[coord[1], coord[0]] for coord in route]

            # Cria a linha da rota
            linha = folium.PolyLine(
                route_lat_lon,
                color='blue',
                weight=1,
                opacity=1
            ).add_to(m)

            # Adiciona setas ao longo da linha para indicar o sentido
            PolyLineTextPath(
                linha,
                text='►',  # Seta Unicode
                repeat=True,
                offset=3,
                attributes={'fill': 'blue', 'font-size': '12', 'letter-spacing': '20'}
            ).add_to(m)

# Exibe o mapa
m.save('rota_otimizada_curitiba_com_arruamentos.html')
print("Mapa salvo como 'rota_otimizada_curitiba_com_arruamentos.html'")

Mapa salvo como 'rota_otimizada_curitiba_com_arruamentos.html'
